In [1]:
import numpy as np
import numpy.linalg as linalg


%run functions.ipynb 

class MDFM():
    def __init__(self, data,  max_lag_to_check):
        self.observations=data
        self.max_lag=max_lag_to_check
        self.left_n_factor=None
        self.right_n_factor=None
        self.M_L=None
        self.M_R=None
        self.left_loading=None
        self.right_loading=None
        self.factors=None
        self.residual=None
    def outer_fit(self):
        def cross_auto_cov(y,k):
            """calculate the auto covariance matrix of columns of matrix time series y with its k-lagged version.
            y is an numpy array of dimension p1*p2*T"""
            mean=y.mean(axis=2)[:, :,np.newaxis]
            y_t=y-mean
            #print(mean)
            T=y.shape[2]
            y_t_plus_k=np.roll(y_t,-k,axis=2)
            y_t=y_t[:,:,:T-k]
            y_t_plus_k=y_t_plus_k[:,:,:T-k]
            return np.einsum('pit,qjt->ijpq',y_t,y_t_plus_k)/(T-k)

        """def construct_M(y,k_0):
            Omega=cross_auto_cov(y,1)
            M=np.einsum('ijpl,ijql->pq',Omega,Omega)
            for k in range(2,k_0+1):
                Omega=cross_auto_cov(y,k)
                M=M+np.einsum('ijpl,ijql->pq',Omega,Omega)
            return M"""
        
        
        def construct_M(y,k_0):
            TOPOP=np.array([cross_auto_cov(y,i) for i in range(k_0+1)])
            mat_TOPOP=ten2mat(TOPOP,3)
            return mat_TOPOP@mat_TOPOP.T
        
        def eigen_decomp(M):
            eigenValues, eigenVectors = linalg.eig(M)

            idx = eigenValues.argsort()[::-1]   
            eigenValues = eigenValues[idx]
            eigenVectors = eigenVectors[:,idx]
            return np.real(eigenValues),np.real(eigenVectors)

        def select_r(eigen_Values):
            #limit=len(eigen_Values)//2+1
            #e_values=eigen_Values[:limit]
            e_values=eigen_Values
            if len(eigenValues)==1:
                return 1
            ratios=np.roll(e_values,-1)[:-1]/e_values[:-1]
            #print(ratios)
            return np.argmin(ratios)+1
        
        self.M_L=construct_M(self.observations,self.max_lag)
        eigenValues,eigenVectors=eigen_decomp(self.M_L)
        self.left_n_factor=select_r(eigenValues)
        self.left_loading=eigenVectors[:,:self.left_n_factor]
        
        self.M_R=construct_M(np.einsum('ijt->jit',self.observations),self.max_lag)
        eigenValues,eigenVectors=eigen_decomp(self.M_R)
        self.right_n_factor=select_r(eigenValues)
        self.right_loading=eigenVectors[:,:self.right_n_factor]
        
        self.factors=np.einsum('ik,klt,lj->ijt',self.left_loading.T,self.observations,self.right_loading)
        self.residual=self.observations-np.einsum('ik,klt,lj->ijt',self.left_loading,self.factors,self.right_loading.T)
        
    def iter_fit(self,max_iter,limit):
        for _ in range(max_iter):
            err=[]
            data=np.einsum('ilt,lj->ijt',self.observations,self.right_loading)
            model=MDFM(data,self.max_lag)
            model.outer_fit()
            err.append(1-trace_stat(model.left_loading.T,self.left_loading.T))
            self.left_loading=model.left_loading
            self.left_n_factor=model.left_n_factor

            data=np.einsum('ik,kjt->ijt',self.left_loading.T,self.observations)
            model=MDFM(data,self.max_lag)
            model.outer_fit()
            err.append(1-trace_stat(model.right_loading.T,self.right_loading.T))
            self.right_loading=model.right_loading
            self.right_n_factor=model.right_n_factor
            
            print(max(err))
            if max(err)<limit:
                break
        self.factors=np.einsum('ik,klt,lj->ijt',self.left_loading.T,self.observations,self.right_loading)
        self.residual=self.observations-np.einsum('ik,klt,lj->ijt',self.left_loading,self.factors,self.right_loading.T)
    def fit(self):
        def cross_auto_cov(y,k):
            """calculate the auto covariance matrix of columns of matrix time series y with its k-lagged version.
            y is an numpy array of dimension p1*p2*T"""
            mean=y.mean(axis=2)[:, :,np.newaxis]
            y_t=y-mean
            #print(mean)
            T=y.shape[2]
            y_t_plus_k=np.roll(y_t,-k,axis=2)
            y_t=y_t[:,:,:T-k]
            y_t_plus_k=y_t_plus_k[:,:,:T-k]
            return np.einsum('ipt,jpt->ij',y_t,y_t_plus_k)/(T-k)

        """def construct_M(y,k_0):
            Omega=cross_auto_cov(y,1)
            for k in range(2,k_0+1):
                Omega=Omega+cross_auto_cov(y,k)
            return np.einsum('ik,jk->ij',Omega,Omega)"""
        
        def construct_M(y,k_0):
            M=cross_auto_cov(y,1)@cross_auto_cov(y,1).T
            for k in range(2,k_0+1):
                M=M+cross_auto_cov(y,k)@cross_auto_cov(y,k).T
            return M

        def eigen_decomp(M):
            eigenValues, eigenVectors = linalg.eig(M)

            idx = eigenValues.argsort()[::-1]   
            eigenValues = eigenValues[idx]
            eigenVectors = eigenVectors[:,idx]
            return np.real(eigenValues),np.real(eigenVectors)

        def select_r(eigen_Values):
            #limit=len(eigen_Values)//2+1
            #e_values=eigen_Values[:limit]
            e_values=eigen_Values
            if len(eigenValues)==1:
                return 1
            ratios=np.roll(e_values,-1)[:-1]/e_values[:-1]
            #print(ratios)
            return np.argmin(ratios)+1
        
        self.M_L=construct_M(self.observations,self.max_lag)
        eigenValues,eigenVectors=eigen_decomp(self.M_L)
        self.left_n_factor=select_r(eigenValues)
        self.left_loading=eigenVectors[:,:self.left_n_factor]
        
        self.M_R=construct_M(np.einsum('ijt->jit',self.observations),self.max_lag)
        eigenValues,eigenVectors=eigen_decomp(self.M_R)
        self.right_n_factor=select_r(eigenValues)
        self.right_loading=eigenVectors[:,:self.right_n_factor]
        
        self.factors=np.einsum('ik,klt,lj->ijt',self.left_loading.T,self.observations,self.right_loading)
        self.residual=self.observations-np.einsum('ik,klt,lj->ijt',self.left_loading,self.factors,self.right_loading.T)
        
    def inner_iter_fit(self,max_iter,limit):
        for _ in range(max_iter):
            err=[]
            data=np.einsum('ilt,lj->ijt',self.observations,self.right_loading)
            model=MDFM(data,self.max_lag)
            model.fit()
            err.append(1-trace_stat(model.left_loading.T,self.left_loading.T))
            self.left_loading=model.left_loading
            self.left_n_factor=model.left_n_factor

            data=np.einsum('ik,kjt->ijt',self.left_loading.T,self.observations)
            model=MDFM(data,self.max_lag)
            model.fit()
            err.append(1-trace_stat(model.right_loading.T,self.right_loading.T))
            self.right_loading=model.right_loading
            self.right_n_factor=model.right_n_factor
            
            #print(max(err))
            if max(err)<limit:
                break
        self.factors=np.einsum('ik,klt,lj->ijt',self.left_loading.T,self.observations,self.right_loading)
        self.residual=self.observations-np.einsum('ik,klt,lj->ijt',self.left_loading,self.factors,self.right_loading.T)